In [32]:
import pandas as pd
import numpy as np
import requests

In [74]:
def paymentCalculator(homePrice, bed, bath, downPaymentPercentage, closingCostPercentage, loanPayments, propertyTaxRate, 
                      HOA, mortgageInsuranceRate, mortgageRate, areaRentalRate):
    downPayment = homePrice*downPaymentPercentage
    closingCosts = (homePrice-downPayment)*closingCostPercentage
    Principal = (homePrice-downPayment)
    propertyTax = (propertyTaxRate*homePrice)/12
    P_I = ((Principal*mortgageRate)/loanPayments)+(Principal/loanPayments)
    mortgagePayment = P_I+propertyTax+HOA
    if downPaymentPercentage <.20:
        mortgageInsurance = (Principal * mortgageInsuranceRate)/12
    else: 
        mortgageInsurance = 0
    mortgagePayment = P_I+propertyTax+HOA+mortgageInsurance
    yearlyCost = mortgagePayment*12
    investmentCosts = {'HomePrice':[homePrice], 'bed':[bed], 'bath':[bath],'downPayment':[downPayment], 'closingCosts':[closingCosts], 
                   'mortgageRate':[mortgageRate], 'loanPayments':[loanPayments], 'Principal':[Principal], 
                   'propertyTax':[propertyTax], 'HOA':[HOA], 'P&I':[P_I], 'mortgageInsurance':[mortgageInsurance],
                       'mortgagePayment':[mortgagePayment], 'yearlyCost':[yearlyCost]}
    global df
    df = pd.DataFrame(data=investmentCosts)
    if (mortgagePayment-areaRentalRate)>0:
        print('Will not cash flow by $', (mortgagePayment-areaRentalRate))
    else:
        print('Will cash flow $', ((mortgagePayment-areaRentalRate)*-1), 'monthly')
        print('Will cash flow $', (((mortgagePayment-areaRentalRate)*12)*-1), 'yearly')
    return df

In [41]:
## Now that we have our function lets get some data, we need the area rental rate
## Comparing to rental rates from 2021 craigslist scrape to see what the average rental rate for our property is
rentalData = pd.read_csv('sdRents.csv')

In [44]:
rentalData.describe()

,Unnamed: 0,Price,Sq_Ft,Bedrooms,Bathrooms,EV Charging,AC,Cats OK,Dogs OK,Furnished,...,wood_floors,carpet_floors,concrete_floors,tile_floors,earthen_floors,other_floors,daily_rent,weekly_rent,monthly_rent,yearly_rent
count,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000,...,2251.000000,2251.000000,2251.000000,2251.000000,2251.0,2251.000000,2251.000000,2251.000000,2251.000000,2251.000000
mean,1440.537983,2566.838738,911.402044,1.624611,1.414038,0.107508,0.224789,0.637050,0.623723,0.051533,...,0.118614,0.050200,0.002221,0.023545,0.0,0.035540,0.000888,0.000444,0.115948,0.109729
std,873.676057,950.288260,412.863165,0.844800,0.561789,0.309827,0.417536,0.480957,0.484559,0.221131,...,0.323406,0.218406,0.047088,0.151661,0.0,0.185181,0.029801,0.021077,0.320234,0.312621
min,0.000000,500.000000,140.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,676.500000,1950.000000,700.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1411.000000,2414.000000,850.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2172.500000,2897.500000,1030.500000,2.000000,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
max,2998.000000,9500.000000,7545.000000,6.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000


In [133]:
bed = 3
bath = 2
a = rentalData[(rentalData['Bedrooms'] == bed) & (rentalData['Bathrooms'] == bath)]
b = int(a['Price'].mean())


In [134]:
paymentCalculator(700000,bed,bath,.05,.03,360,.0125,300,.01,.065, b)

Will cash flow $ 55.375000000000455 monthly
Will cash flow $ 664.5000000000055 yearly
$ 197 is the amount of home price per dollar of payment
710382 is area rental rate home value break even


,HomePrice,bed,bath,downPayment,closingCosts,mortgageRate,loanPayments,Principal,propertyTax,HOA,P&I,mortgageInsurance,mortgagePayment,yearlyCost
0,700000,3,2,35000.0,19950.0,0.065,360,665000.0,729.166667,300,1967.291667,554.166667,3550.625,42607.5


In [135]:
## Given our rental is non cashflowing, lets now check to see what the home price would need to be to make sense to buy given  the same rate, and other variables
## Get the rate of home price to payment
c = df['HomePrice']/df['mortgagePayment']
c

0    197.148389
dtype: float64

In [131]:
## Updating function to get home price to payment

def paymentCalculator(homePrice, bed, bath, downPaymentPercentage, closingCostPercentage, loanPayments, propertyTaxRate, 
                      HOA, mortgageInsuranceRate, mortgageRate, areaRentalRate):
    downPayment = homePrice*downPaymentPercentage
    closingCosts = (homePrice-downPayment)*closingCostPercentage
    Principal = (homePrice-downPayment)
    propertyTax = (propertyTaxRate*homePrice)/12
    P_I = ((Principal*mortgageRate)/loanPayments)+(Principal/loanPayments)
    mortgagePayment = P_I+propertyTax+HOA
    if downPaymentPercentage <.20:
        mortgageInsurance = (Principal * mortgageInsuranceRate)/12
    else: 
        mortgageInsurance = 0
    mortgagePayment = P_I+propertyTax+HOA+mortgageInsurance
    yearlyCost = mortgagePayment*12
    investmentCosts = {'HomePrice':[homePrice], 'bed':[bed], 'bath':[bath],'downPayment':[downPayment], 'closingCosts':[closingCosts], 
                   'mortgageRate':[mortgageRate], 'loanPayments':[loanPayments], 'Principal':[Principal], 
                   'propertyTax':[propertyTax], 'HOA':[HOA], 'P&I':[P_I], 'mortgageInsurance':[mortgageInsurance],
                       'mortgagePayment':[mortgagePayment], 'yearlyCost':[yearlyCost]}
    global df
    df = pd.DataFrame(data=investmentCosts)
    a = int(df['HomePrice']/df['mortgagePayment'])
    if (mortgagePayment-areaRentalRate)>0:
        print('Will not cash flow by $', (mortgagePayment-areaRentalRate))
        print("$", a, "is the amount of home price per dollar of payment")
        print((areaRentalRate*a), "is area rental rate home value break even")
    else:
        print('Will cash flow $', ((mortgagePayment-areaRentalRate)*-1), 'monthly')
        print('Will cash flow $', (((mortgagePayment-areaRentalRate)*12)*-1), 'yearly')
        print("$", a, "is the amount of home price per dollar of payment")
        print((areaRentalRate * a), "is area rental rate home value break even")
    return df

In [132]:
paymentCalculator(407000,bed,bath,.05,.03,360,.0125,300,.01,.065, b)

Will not cash flow by $ 0.0062499999999090505
$ 185 is the amount of home price per dollar of payment
405150 is area rental rate home value break even


,HomePrice,bed,bath,downPayment,closingCosts,mortgageRate,loanPayments,Principal,propertyTax,HOA,P&I,mortgageInsurance,mortgagePayment,yearlyCost
0,407000,2,1,20350.0,11599.5,0.065,360,386650.0,423.958333,300,1143.839583,322.208333,2190.00625,26280.075


In [ ]:
## With single property estimation completed next up is ML 